In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel

# 📄 Excel-Datei laden
df = pd.read_excel("UX-Daten.xlsx", sheet_name="Formularantworten 1")

# 🧾 Teilnehmer-ID hinzufügen
df.insert(0, "TeilnehmerID", range(1, len(df) + 1))

# 🎯 Bewertete Spalten identifizieren
value_cols = [col for col in df.columns if col.startswith("AR ") or col.startswith("Bild ")]

# 🔄 Long-Format erzeugen
df_long = df.melt(
    id_vars=["TeilnehmerID"],
    value_vars=value_cols,
    var_name="ItemBedingung",
    value_name="Wert"
)

# 📌 Bedingung extrahieren
df_long["Bedingung"] = df_long["ItemBedingung"].apply(lambda x: "AR" if x.startswith("AR ") else "Bild")

# 📌 Item bereinigen (z. B. 'AR hässlich – schön (ATT)' → 'hässlich – schön (ATT)')
df_long["Item"] = df_long["ItemBedingung"].str.replace(r"^(AR |Bild )", "", regex=True)

# 📌 UX-Dimension aus Klammerinhalt extrahieren → 'PQ', 'HQ-S', 'HQ-I', 'ATT'
df_long["Dimension"] = df_long["Item"].str.extract(r"\((.*?)\)$")[0]


In [2]:
# Auswertung pro Item
# Pivotieren für gepaarten t-Test
pivot_items = df_long.pivot(index=["TeilnehmerID", "Item"], columns="Bedingung", values="Wert").dropna()

# t-Test + Effektstärke je Item
results_items = []
for item in pivot_items.index.get_level_values("Item").unique():
    ar = pivot_items.xs(item, level="Item")["AR"]
    bild = pivot_items.xs(item, level="Item")["Bild"]
    t, p = ttest_rel(ar, bild)
    d = (ar.mean() - bild.mean()) / np.sqrt(((ar.std(ddof=1)**2 + bild.std(ddof=1)**2)/2))
    results_items.append({
        "Item": item,
        "Mittelwert_AR": ar.mean(),
        "Mittelwert_Bild": bild.mean(),
        "t-Wert": t,
        "p-Wert": p,
        "Cohen's d": d
    })

df_results_items = pd.DataFrame(results_items).sort_values("p-Wert")


In [3]:
# Mittelwerte pro Teilnehmer x Dimension x Bedingung
df_mean_dim = df_long.groupby(["TeilnehmerID", "Dimension", "Bedingung"])["Wert"].mean().reset_index()

# Pivotieren für Dimensionen
pivot_dim = df_mean_dim.pivot(index=["TeilnehmerID", "Dimension"], columns="Bedingung", values="Wert").dropna()

# t-Test + Effektstärke je Dimension
results_dims = []
for dim in pivot_dim.index.get_level_values("Dimension").unique():
    ar = pivot_dim.xs(dim, level="Dimension")["AR"]
    bild = pivot_dim.xs(dim, level="Dimension")["Bild"]
    t, p = ttest_rel(ar, bild)
    d = (ar.mean() - bild.mean()) / np.sqrt(((ar.std(ddof=1)**2 + bild.std(ddof=1)**2)/2))
    results_dims.append({
        "Dimension": dim,
        "Mittelwert_AR": ar.mean(),
        "Mittelwert_Bild": bild.mean(),
        "t-Wert": t,
        "p-Wert": p,
        "Cohen's d": d
    })

df_results_dims = pd.DataFrame(results_dims).sort_values("p-Wert")


In [4]:
# 📁 Exportiere Item-basierte Ergebnisse
df_results_items.to_excel("auswertung_items.xlsx", index=False)

# 📁 Exportiere UX-Dimensions-Ergebnisse
df_results_dims.to_excel("auswertung_dimensionen.xlsx", index=False)
